# 붓꽃의 품종 분류

- 어떤 품종인지 구분해놓은 측정 데이터를 이용해 새로 채집한 붓꽃의 품종을 에측하는 머신러닝 모델을 만듦
- 붓꽃의 품종(class): setosa, versicolor, virginica
- 지도학습, 분류(classification) 문제에 해당

## Scikit-learn에 있는 데이터를 이용

### 1. 붓꽃 데이터셋 불러오기

- scikit-lear의 dataset 모듈에 포함
- load_iris 함수를 사용

In [25]:
from sklearn.datasets import load_iris
irisData = load_iris() # bunch class 객체형태로 반환, dictionary와 유사, 키와 값으로 구성

#### 1) iris 데이터셋의 key

In [10]:
irisData.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

#### 2) iris 데이터셋의 설명

In [11]:
irisData['DESCR'][:200]

'Iris Plants Database\n====================\n\nNotes\n-----\nData Set Characteristics:\n    :Number of Instances: 150 (50 in each of three classes)\n    :Number of Attributes: 4 numeric, predictive attributes'

#### 3) target name: 예측하려는 붓꽃 품종의 이름(label)

In [8]:
irisData['target_names']

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

#### 4) feature name: 특성 이름

In [9]:
irisData['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

#### 5) 실제 데이터: 꽃받침(sepal)의 길이와 폭, 꽃잎(petal)의 길이와 폭

In [19]:
irisData['data'][:10]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

In [20]:
type(irisData['data'])

numpy.ndarray

In [21]:
irisData['data'].shape

(150, 4)

In [22]:
irisData['target']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [23]:
irisData['target'].shape

(150,)

### 2. Data를 training set과 test set으로 나누기

- 모델을 만들 때 쓴 데이터는 평가 목적으로 사용 불가
- scikit-learn에서 data는 대문자 X로 표기, label은 소문자 y로 표기
- scikit-learn은 데이셋을 섞어서 나눠주는 train_test_split 함수를 제공
    - train_test_split 모듈에 있는 train_test_split()
    - train_test_split 모듈은 sklearn.model_selection에 있음
    - train_test_split()의 리턴타입은 모두 numpy 배열
    - 전체 행 중 75%를 레이블 데이터와 함꼐 훈련 세트로 뽑음, 나머지 25%는 레이블과 함께 테스트 세트(기본값)
        - test_size 매개변수로 비율 지정 가능
- train_test_split으로 데이터를 나누기 전에 유사난수생성기를 이용하여 데이터 셋을 무작위로 섞는 것이 필요
    - 한 레이블 값만 같은 데이터를 방지하기 위함
    - random_state=0
        - 함수를 여러번 실행해도 결과가 똑같이 나오도록 고정

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(irisData['data'], 
                                                    irisData['target'],
                                                   random_state=0)

- 자동으로 X_train, x_test가 75:25로 나눠짐 

In [30]:
X_train.shape

(112, 4)

In [31]:
X_test.shape

(38, 4)

In [32]:
y_train.shape

(112,)

In [33]:
y_test.shape

(38,)

### 3. KNN 알고리즘을 이용하여 머신러닝 모델생성, 예측, 평가

- KNN(K-Nearest Neighbors, 최근접 이웃) 알고리즘 사용
    - SVM과 더불어 분류 모델에서 사용되는 알고리즘
    - 새로운 데이터 포인트에 대한 예측이 필요한 경우 새 데이터 포인트에서 가장 가까운 훈련데이터 포인트를 찾음
    - k는 훈련데이터에서 새로운 데이터 포인트에 가장 가까운 k개의 이웃을 찾는 것을 의미

#### 1) knn 객체 만들기 

- Neighbors 모듈 아래 KNeighborsClassifier 클래스에 구현됨
- n_neighbors 매개변수를 통해 k의 개수, 즉 이웃의 개수를 지정

In [56]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)

#### 2) 모델 생성, 학습

- 훈련데이터셋을 가지고 모델을 만들기 위해서 fit 메서드 사용

In [57]:
# fit 메서드의 return값은 knn 객체 리턴
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

#### 3) 예측

In [58]:
# 채집한 붓꽃의 새로운 데이터(샘플, 데이터 포인터)라고 가정하고
# Numpy배열로 특성값을 만듦
# scikit-learn에서는 일반적으로 데이터가 2차원 배열일 것으로 예측
X_newData = np.array([[5, 2.9, 1, 0.2]])
X_newData

array([[5. , 2.9, 1. , 0.2]])

In [59]:
X_newData.shape

(1, 4)

In [64]:
prediction = knn.predict(X_newData)
print("predict: {}".format(prediction))
print("predicted target name: {}".format(irisData['target_names'][prediction]))

predict: [0]
predicted target name: ['setosa']


#### 4) 평가

- 참고: np.mean

In [74]:
x = np.array([1, 2, 3, 2])
print("max value in x: {}".format(x.max()))
print("min value in x: {}".format(x.min()))
print("sum value in x: {}".format(x.sum()))
print("mean value in x: {}".format(x.mean()))

max value in x: 3
min value in x: 1
sum value in x: 8
mean value in x: 2.0


In [78]:
a = np.array([1, 2, 3])
b = np.array([1, 2, 3])
print("a==b: {}".format(a==b))
print("np.mean(a==b): {}".format(np.mean(a==b)))

a==b: [ True  True  True]
np.mean(a==b): 1.0


In [81]:
c = np.array([1, 2, 3, 4])
d = np.array([1, 1, 1, 4])
print("c==d: {}".format(c==d))
print("np.mean(c==d): {}".format(np.mean(c==d)))

c==d: [ True False False  True]
np.mean(c==d): 0.5


##### i. 평균값을 이용하여 정확도 계산

- 테스트 데이터 셋을 에측, 테스트 레이블과 비교하여 모델 평가

In [66]:
y_predict = knn.predict(X_test)
y_predict

array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 0, 2])

In [88]:
print("test set accuracy: {:.2f}".format(np.mean(y_predict == y_test)))

test set accuracy: 0.97


##### ii. knn.score 메서드를 이용하여 정확도 계산

In [87]:
knn.score(X_test, y_test)

0.9736842105263158

## iris.csv 파일을 이용

### 1. 데이터 불러오기

- 데이터는 구글에서 iris.csv로 검색하여 다운로드 함
- [데이터 다운로드](https://github.com/pandas-dev/pandas/raw/master/pandas/tests/data/iris.csv)

In [111]:
import pandas as pd
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split

In [109]:
data_url = 'https://github.com/pandas-dev/pandas/raw/master/pandas/tests/data/iris.csv'
iris_df = pd.read_csv(data_url)
iris_df.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [110]:
iris_df.tail()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


### 2. Sample Data를 Feature Data와 Target(label)로 분할

In [122]:
iris_data = iris_df[['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']]
iris_data.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [124]:
iris_target = iris_df['Name']
iris_target.head()

0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: Name, dtype: object

In [125]:
data = iris_data.as_matrix()
data.shape

(150, 4)

In [126]:
data[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [127]:
target = iris_target.as_matrix()
target.shape

(150,)

In [130]:
target[:5]

array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa'], dtype=object)

### 3. Data를 training set과 test set으로 나누기

In [138]:
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 1/3,
                                                    random_state=0)

In [139]:
X_train.shape

(100, 4)

In [142]:
X_test.shape

(50, 4)

In [144]:
y_train.shape

(100,)

In [145]:
y_test.shape

(50,)

### 4. SVM 알고리즘을 이용하여 머신러닝 모델생성, 예측, 평가

#### 1) SVM 객체 만들기, 모델생성(학습) 

In [146]:
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

#### 2) 예측

In [162]:
y_pred = clf.predict(X_test)
print("predict: {}".format(y_pred))

predict: ['Iris-virginica' 'Iris-versicolor' 'Iris-setosa' 'Iris-virginica'
 'Iris-setosa' 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-virginica' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa' 'Iris-setosa'
 'Iris-virginica' 'Iris-versicolor' 'Iris-setosa' 'Iris-setosa'
 'Iris-virginica' 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-setosa' 'Iris-virginica' 'Iris-versicolor'
 'Iris-setosa' 'Iris-virginica' 'Iris-virginica' 'Iris-versicolor'
 'Iris-setosa' 'Iris-virginica' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-virginica' 'Iris-setosa' 'Iris-virginica' 'Iris-setosa'
 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica'
 'Iris-virginica' 'Iris-virginica']


In [163]:
y_pred.shape

(50,)

#### 3) 평가

In [165]:
accuracy = metrics.accuracy_score(y_test, y_pred);
print("accuracy: {}".format(accuracy))

accuracy: 0.98
